In [ ]:
import pandas as pd
import dask.dataframe as dd
#from dask.distributed import Client
import numpy as np 
import os
import glob
#import psycopg2
import warnings

warnings.filterwarnings('ignore')

* Create several lookup tables for a unified spatial scale at either 2021 LSOAs or 2011 DZs

In [ ]:
# Read a lookup table from ONSPD UK 2023.11 
lookup = pd.read_csv('Raw data/ONSPD_NOV_2023_UK.csv')
lookup = lookup[['pcd','oa21','lsoa21','msoa21', 'oslaua','rgn','oa11','lsoa11','msoa11']].fillna('NA')
# England and Wales 
lookup_EW = lookup[lookup['lsoa21'].str.contains('E|W')]
# Scotland only
lookup_S = lookup[lookup['lsoa11'].str.startswith('S0')]


In [ ]:
# Convert all varying spatial scales from household and 2011 LSOAs to 2021 LSOAs/2011 DZs

# postcode to LSOA21
pcd_lsoa21 = lookup_EW[['pcd','lsoa21']]
pcd_dz11 = lookup_S[['pcd','lsoa11']]
pcd_lsoa21.columns = ['POSTCODE','lsoa21']
pcd_dz11.columns = ['POSTCODE','lsoa21']
pcd_lsoa21_gb = pd.concat([pcd_lsoa21,pcd_dz11])
pcd_lsoa21_gb['POSTCODE'] = pcd_lsoa21_gb['POSTCODE'].str.replace(' ','')

# LSOA11 to LSOA21
lsoa11_21 = lookup_EW[['lsoa21','lsoa11']].drop_duplicates().set_index('lsoa11')
#create a weight variable: divisions based on the count of LSOA11 at each LSOA11 group
lsoa11_21 = lsoa11_21.join(1/lsoa11_21.groupby('lsoa11').count().rename(columns = {'lsoa21':'weight'})).sort_values('weight')

* Energy Efficiency data for England and Wales as well as Scotland

In [ ]:
#EPC for England and Wales
dir1 = 'Raw data\\EPC_EW\\all-domestic-certificates(2008Q4-2023Q3)\\'
csvfiles = []
for root,dirs,files in os.walk(dir1):
    for file in files:
        if file.endswith('certificates.csv'):
            df = dd.read_csv(root + '\\'+'certificates.csv',sample = 10000,
        dtype={'ENERGY_CONSUMPTION_POTENTIAL': 'float64',
       'HEATING_COST_CURRENT': 'float64',
       'HEATING_COST_POTENTIAL': 'float64',
       'HOT_WATER_COST_CURRENT': 'float64',
       'HOT_WATER_COST_POTENTIAL': 'float64',
       'LIGHTING_COST_CURRENT': 'float64',
       'LIGHTING_COST_POTENTIAL': 'float64',
        'WIND_TURBINE_COUNT': 'float64',
       'MAIN_HEATING_CONTROLS': 'object',
               'CONSTITUENCY': 'object',
       'CONSTITUENCY_LABEL': 'object',
               'ADDRESS2':'object',
              'ADDRESS3': 'object',
               'COUNTY': 'object',
               'UPRN':'object',
               'FLOOR_ENV_EFF':'object','FLOOR_ENERGY_EFF':'object',
              'FLAT_TOP_STOREY': 'object','HEAT_LOSS_CORRIDOR': 'object'},assume_missing = True)
            #select useful attributes for this study
            df = df.iloc[:,[4,6,8,16,18,22,23,31,33,-19,-7,-2]]
            #dask dataframe to pandas dataframe
            csvfiles.append(df.compute())

In [ ]:
epc = pd.concat(csvfiles)
#EPC with uprn
epc_uprn = epc[~epc['UPRN'].isnull()]
epc_uprn['LODGEMENT_DATE'] = epc_uprn['LODGEMENT_DATE'].str.replace('-','')
epc_uprn['YEAR-MONTH'] = epc_uprn['LODGEMENT_DATE'].str[0:6].astype('int')
# analysis only includes the latest EPC lodgements for a 10 year period, from Q4 2013 to Q3 2023.
epc_uprn = epc_uprn[(epc_uprn['YEAR-MONTH']>201309) & (epc_uprn['YEAR-MONTH']<202310)]
# Only count the latest valid EPC variable for each dwelling/property to avoid duplicate counts, so only one valid epc for each dwelling
valid_epc = epc_uprn.sort_values(['UPRN','YEAR-MONTH']).drop_duplicates(['UPRN','POSTCODE'],keep = 'last')
valid_epc = valid_epc[['UPRN','POSTCODE','CURRENT_ENERGY_EFFICIENCY','CO2_EMISSIONS_CURRENT','TOTAL_FLOOR_AREA',\
                'CO2_EMISS_CURR_PER_FLOOR_AREA','CONSTRUCTION_AGE_BAND','MAIN_FUEL']]
valid_epc

In [ ]:
# EPC for Scotland
path = glob.glob('Raw data/EPC_S/D_EPC_data_2012Q4-2023Q3/*.csv')
files = [pd.read_csv(f, header = 1) for f in path]
epc_S = pd.concat(files)
epc_S = epc_S[['OSG_UPRN', 'Postcode','Current energy efficiency rating','Current Emissions (T.CO2/yr)','Total floor area (m²)',\
      'CO2 Emissions Current Per Floor Area (kg.CO2/m²/yr)','Part 1 Construction Age Band','Main Heating 1 Fuel Type','Date of Certificate']]
epc_S.columns = ['UPRN','POSTCODE','CURRENT_ENERGY_EFFICIENCY','CO2_EMISSIONS_CURRENT','TOTAL_FLOOR_AREA',\
                'CO2_EMISS_CURR_PER_FLOOR_AREA','CONSTRUCTION_AGE_BAND','MAIN_FUEL','Date of Certificate']
#drop records without a single UPRN
epc_uprn_S = epc_S[~epc_S['UPRN'].isnull()]
# Only count the latest valid EPC variable for each dwelling/property to avoid duplicate counts, so only one valid epc for each dwelling
valid_epc_S = epc_uprn_S.sort_values(['UPRN','Date of Certificate']).drop_duplicates(['UPRN','POSTCODE'],keep = 'last')

In [ ]:
# merge EPC data togother for GB 
valid_epc_gb = pd.concat([valid_epc, valid_epc_S.drop('Date of Certificate', axis = 1)])

* Using EPC to measure variables within Energy Efficiency domain

In [ ]:
# energy efficiency Band D and below
valid_epc_gb['Efficiency band E-G'] = np.where(valid_epc_gb['CURRENT_ENERGY_EFFICIENCY'] < 55, 'Y','N')

# energy efficiency Band C and above
valid_epc_gb['Efficiency band A-B'] = np.where(valid_epc_gb['CURRENT_ENERGY_EFFICIENCY'] > 80, 'Y','N')

# main fuel type is fossil fuel reliances
valid_epc_gb['MAIN_FUEL'] = valid_epc_gb['MAIN_FUEL'].fillna('NA')
valid_epc_gb['Fossil fuel dependency'] = np.where((valid_epc_gb['MAIN_FUEL']=='oil (not community)') | (valid_epc_gb['MAIN_FUEL']=='oil (community)') | (valid_epc_gb['MAIN_FUEL']=='Oil: heating oil') |\
                                              (valid_epc_gb['MAIN_FUEL'].str.contains('mains gas')) | (valid_epc_gb['MAIN_FUEL'].str.contains('LPG'))|(valid_epc_gb['MAIN_FUEL'].str.contains('LNG'))|\
                                               (valid_epc_gb['MAIN_FUEL'].str.contains('coal')) | (valid_epc_gb['MAIN_FUEL']=='anthracite'), 'Y','N')
valid_epc_gb['Fossil fuel dependency'] = np.where(valid_epc_gb['MAIN_FUEL'].str.contains('for backwards compatibility only'), 'N',valid_epc_gb['Fossil fuel dependency'])

# CO2 emission per square meter floor area km2/year, mean CO2 emission per square meter km2/year is 45.7
valid_epc_gb['CO2_EMISSIONS_CURRENT'] = np.where(valid_epc_gb['CO2_EMISSIONS_CURRENT']<0, np.nan,valid_epc_gb['CO2_EMISSIONS_CURRENT'])

# property age band before 2012
valid_epc_gb['CONSTRUCTION_AGE_BAND'] = valid_epc_gb['CONSTRUCTION_AGE_BAND'].fillna('NA')
valid_epc_gb['Property built 1930-'] = np.where((valid_epc_gb['CONSTRUCTION_AGE_BAND']=='England and Wales: before 1900') | (valid_epc_gb['CONSTRUCTION_AGE_BAND']=='England and Wales: 1900-1929') |\
                                                (valid_epc_gb['CONSTRUCTION_AGE_BAND']=='1919-1929') | (valid_epc_gb['CONSTRUCTION_AGE_BAND']=='before 1919')|\
                                 (valid_epc_gb['CONSTRUCTION_AGE_BAND'].str.contains('1880|1800|1890|1850|1889|1820|1885|1825|1876|1849|1900|1902|1910|1920|1929')), 'Y','N')
valid_epc_gb['Property built 2003+'] = np.where((valid_epc_gb['CONSTRUCTION_AGE_BAND']=='England and Wales: 2012 onwards') |\
                                                (valid_epc_gb['CONSTRUCTION_AGE_BAND']=='England and Wales: 2007-2011')|\
                                                (valid_epc_gb['CONSTRUCTION_AGE_BAND']=='England and Wales: 2007 onwards')|\
                                                (valid_epc_gb['CONSTRUCTION_AGE_BAND']=='England and Wales: 2003-2006')|\
                                                (valid_epc_gb['CONSTRUCTION_AGE_BAND']== '2008 onwards')|(valid_epc_gb['CONSTRUCTION_AGE_BAND']== '2003-2007')|\
                                 (valid_epc_gb['CONSTRUCTION_AGE_BAND'].str.contains('2003|2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021|2022|2023')), 'Y','N')

In [ ]:
epc_df = valid_epc_gb[['POSTCODE','Efficiency band E-G','Efficiency band A-B','Fossil fuel dependency','Property built 1930-','Property built 2003+','CO2_EMISSIONS_CURRENT']]
#merge with lookup table 
epc_df['POSTCODE'] = epc_df['POSTCODE'].str.replace(' ','')
epc_df = epc_df.merge(pcd_lsoa21_gb, on = 'POSTCODE')

* Proportional measures of variables from EPC

In [ ]:
def proportion_efficiency(data,col):
    #count the total number of EPC records at each LSOA21
    epc_sum_lsoa = data.groupby('lsoa21').count()[['POSTCODE']]
    #count when feature is'Y'
    count = data[data[data.columns[col]] == 'Y'].groupby('lsoa21').count()[[data.columns[col]]]
    #join total count and feature count
    count = count.join(epc_sum_lsoa)
    #suppress count lower than 5 as 0, which are not deemed enough to produce a robust statistic.
    count[count.columns[0]]  = np.where(count[count.columns[0]] < 5, 0, count[count.columns[0]])
    #calculate the proportion
    count[count.columns[0]] = (count[count.columns[0]]/count['POSTCODE']*100).round(2)

    return count

In [ ]:
epc_vars = pd.concat([proportion_efficiency(epc_df,i).iloc[:,[0]] for i in range(1,epc_df.shape[1]-1)], axis = 1)
# merge with CO2 emission
epc_vars['CO2_EMISSIONS_CURRENT'] = epc_df.groupby('lsoa21').mean()

In [ ]:
epc_vars.to_csv('Vars_EPC.csv')